In [10]:
import datasets
from datasets import load_dataset
train_dataset = load_dataset("sadassa17/rgb-spatial-dataset", split='train', streaming=True)
print(next(iter(train_dataset)))

{'image': '<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7FB9FDBD2890>', 'caption': 'The bowl is bellow the bowl. The bowl is above the bowl. The bowl is in front of the bowl. The bowl is behind the bowl. The broccoli is bellow the bowl. The bowl is above the broccoli. The broccoli is in front of the bowl. The bowl is behind...', 'image_path': 'resized_train/COCO_train2014_000000000009.jpg', 'height': 224, 'width': 224, 'image_id': 0, 'file_name': 'COCO_train2014_000000000009'}


In [11]:
import datasets
from datasets import load_dataset
val_dataset = load_dataset("sadassa17/rgb-spatial-dataset", split='validation', streaming=True)
print(next(iter(val_dataset)))

{'Unnamed: 0': 0, 'image': '<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=224x224 at 0x7FB9E7D50730>', 'caption': 'The dog is to the right of the sports ball. The sports ball is to the left of the dog.', 'image_path': 'resized_val/COCO_val2014_000000000042.jpg', 'height': 224, 'width': 224, 'image_id': 0, 'file_name': 'COCO_val2014_000000000042'}


In [12]:
print(type(train_dataset))

<class 'datasets.iterable_dataset.IterableDataset'>


In [1]:
####CODE TO CREATE A LIST OF TENSORS FOR THE WHOLE TRAINING DATASET####
import torch
loaded_tensor1 = torch.load('f16train1_tensor_data.pt')
loaded_tensor2 = torch.load('f16train2_tensor_data.pt')
loaded_tensor3 = torch.load('f16train3_tensor_data.pt')
loaded_tensor4 = torch.load('full_training_tensor.pt')
chunk_size = 1000  # Adjust the chunk size based on available memory
num_chunks = len(loaded_tensor1) // chunk_size
chunk_tensors = []  # List to store individual chunks
for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk_tensor = torch.cat((loaded_tensor1[start_idx:end_idx], loaded_tensor2[start_idx:end_idx], loaded_tensor3[start_idx:end_idx]), dim=0)
    chunk_tensors.append(chunk_tensor)  # Store the chunk_tensor in the list for later use


# If there are remaining elements, create a final chunk for them
if num_chunks * chunk_size < len(loaded_tensor1):
    remaining_tensor = torch.cat((loaded_tensor1[num_chunks * chunk_size:], 
                                 loaded_tensor2[num_chunks * chunk_size:], 
                                 loaded_tensor3[num_chunks * chunk_size:]), dim=0)
    chunk_tensors.append(remaining_tensor)

torch.save(chunk_tensors, 'tensor_list.pth')    

In [5]:
###CODE TO SAVE THE TENSOR AFTER CONVERTING FROM A LIST OF TENSOR TO ONE COMPLETE TENSOR 
# tensor = torch.cat(chunk_tensors, dim=0)
# tensor.size()
# file_name = 'full_training_tensor.pt'
# torch.save(tensor, file_name)

In [9]:
#tensor.size()

torch.Size([82611, 1, 3, 224, 224])

In [ ]:
full_tensor = torch.cat((loaded_tensor1, loaded_tensor2), dim=0)
file_name = f'{full_tensor}_tensor_data.pt'

In [2]:
loaded_tensor3 = torch.load('train3_tensor_data.pt')

NameError: name 'torch' is not defined

In [ ]:
full_tensor = torch.cat((loaded_tensor1, loaded_tensor2), dim=0)
file_name = f'{full_tensor}_tensor_data.pt'
torch.save(full_tensor, file_name)

In [13]:
import os
import datasets
from transformers import VisionEncoderDecoderModel, AutoFeatureExtractor,AutoTokenizer
os.environ["WANDB_DISABLED"] = "true"

In [14]:
import nltk
try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    nltk.download("punkt", quiet=True)


In [15]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor, AutoImageProcessor

image_encoder_model = "Centaur31/vit-base"
text_decode_model = "gpt2"

model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    image_encoder_model, text_decode_model)


Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.3.crossattention.c_attn.weight', 'h.7.ln_cross_attn.weight', 'h.3.crossattention.q_attn.bias', 'h.3.ln_cross_attn.bias', 'h.7.crossattention.q_attn.bias', 'h.0.crossattention.c_attn.weight', 'h.6.crossattention.c_proj.weight', 'h.8.ln_cross_attn.weight', 'h.0.crossattention.q_attn.bias', 'h.7.ln_cross_attn.bias', 'h.5.crossattention.c_attn.bias', 'h.2.crossattention.q_attn.bias', 'h.6.ln_cross_attn.weight', 'h.10.crossattention.c_attn.weight', 'h.11.ln_cross_attn.weight', 'h.2.crossattention.c_proj.weight', 'h.4.ln_cross_attn.bias', 'h.11.crossattention.c_proj.bias', 'h.7.crossattention.c_attn.bias', 'h.4.crossattention.c_attn.weight', 'h.8.ln_cross_attn.bias', 'h.9.crossattention.q_attn.bias', 'h.11.crossattention.c_attn.bias', 'h.5.ln_cross_attn.weight', 'h.2.crossattention.c_proj.bias', 'h.8.crossattention.c_attn.bias', 'h.5.crossattention.c_attn.weight', 'h.4.crossa

In [16]:
# image feature extractor
feature_extractor = AutoImageProcessor.from_pretrained(image_encoder_model)
# text tokenizer
tokenizer = AutoTokenizer.from_pretrained(text_decode_model)

# GPT2 only has bos/eos tokens but not decoder_start/pad tokens
tokenizer.pad_token = tokenizer.eos_token

# update the model config
model.config.eos_token_id = tokenizer.eos_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

model.encoder.embeddings.patch_embeddings.projection

output_dir = "vit-gpt-model"
model.save_pretrained(output_dir)
feature_extractor.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('vit-gpt-model/tokenizer_config.json',
 'vit-gpt-model/special_tokens_map.json',
 'vit-gpt-model/vocab.json',
 'vit-gpt-model/merges.txt',
 'vit-gpt-model/added_tokens.json',
 'vit-gpt-model/tokenizer.json')

In [17]:
import os

def list_items_in_folder(folder_path):
    items_list = []

    # Iterate over each item (files or directories) in the folder
    for item in os.listdir(folder_path):
        # Get the full path of the item
        item_path = os.path.join(folder_path, item)
        # Append the item's path to the list
        items_list.append(item_path)

    return items_list

# Replace 'folder_path' with the path to the folder you want to read
folder_path = 'resized_train'
items_list = list_items_in_folder(folder_path)
items_list = sorted(items_list)

# Replace 'folder_path' with the path to the folder you want to read
val_folder_path = 'resized_val'
val_items_list = list_items_in_folder(val_folder_path)
val_items_list=sorted(val_items_list)

###CODING TO RESIZE IMAGE DATASETS BEFORE PROCESSING####
# # List to store resized images
# resized_images = []
# from PIL import Image
# save_directory = "resized_train"
# if not os.path.exists(save_directory):
#     os.makedirs(save_directory)
# # Resize images and append them to the resized_images list
# for image_path in items_list:
#     # Open the original image using PIL
#     original_image = Image.open(image_path)
#     resized_image = original_image.resize((224, 224))
#     file_name = os.path.splitext(os.path.basename(image_path))[0]
#     new_file_name = f"{file_name}.jpg"
#     save_path = os.path.join(save_directory, new_file_name)
#     resized_image.save(save_path)
#     original_image.close()
#     resized_image.close()


#from datasets import load_dataset, Image, Dataset
from datasets import Image as dataset_Image, Dataset
data = {"image": items_list}
# Step 3: Convert the list to a Dataset object
dataset = Dataset.from_dict(data)
# Step 4: Cast the "image" column to the Image() type
dataset = dataset.cast_column("image", dataset_Image())

#EVALUATION
val_data = {"image": val_items_list}
#Convert the list to a Dataset object
val_dataset = Dataset.from_dict(val_data)

#Cast the "image" column to the Image() type
val_dataset = val_dataset.cast_column("image", dataset_Image())

In [6]:
#EVALUATION
val_heights_list = []
val_widths_list = []
val_image_id_counter = 0
val_text_id_counter = 0
val_image_id_list = []
val_text_id_list = []
# Loop through the 'image' column of the dataset
for image in val_dataset['image']:
    # Get the height and width of the current image
    val_height, val_width = image.size

    # Append the height and width to their respective lists
    val_heights_list.append(val_height)
    val_widths_list.append(val_width)
    val_image_id = val_image_id_counter
    val_image_id_list.append(val_image_id)
    val_image_id_counter += 1

heights_list = []
widths_list = []
image_id_counter = 0
text_id_counter = 0
image_id_list = []
text_id_list = []
#Loop through the 'image' column of the dataset
for image in dataset['image']:
    # Get the height and width of the current image
    height, width = image.size

    # Append the height and width to their respective lists
    heights_list.append(height)
    widths_list.append(width)
    image_id = image_id_counter
    image_id_list.append(image_id)
    image_id_counter += 1

In [7]:
# import tqdm
# val_heights_list = []
# val_widths_list = []
# val_image_id_counter = 0
# val_text_id_counter = 0
# val_image_id_list = []
# val_text_id_list = []

# with tqdm(desc="Processing validation Images") as progress_bar:
# # Loop through the 'image' column of the dataset
#     for image in val_dataset['image']:
#         # Get the height and width of the current image
#         val_height, val_width = image.size

#         # Append the height and width to their respective lists
#         val_heights_list.append(val_height)
#         val_widths_list.append(val_width)
#         val_image_id = val_image_id_counter
#         val_image_id_list.append(val_image_id)
#         val_image_id_counter += 1

#         # Update the tqdm progress bar
#         progress_bar.update(1)

     

# heights_list = []
# widths_list = []
# image_id_counter = 0
# text_id_counter = 0
# image_id_list = []
# text_id_list = []

# progress_bar = tqdm(desc="Processing Training Images")
# for image in dataset['image']:
#     # Get the height and width of the current image
#     height, width = image.size

#     # Append the height and width to their respective lists
#     heights_list.append(height)
#     widths_list.append(width)
#     image_id = image_id_counter
#     image_id_list.append(image_id)
#     image_id_counter += 1

#     # Update the tqdm progress bar
#     progress_bar.update(1)

# # Close the tqdm progress bar after the loop is finished
# progress_bar.close()

In [8]:
import re
def remove_numbers(text_descriptions):
    clean_text_descriptions = []
    for line in text_descriptions:
        clean_text_descriptions.append((re.sub(r'\d+', '', line)).strip())  # Remove digits and leading/trailing spaces
    combined_text = '. '.join(clean_text_descriptions) + '.'  # Add a period at the end
    return combined_text

def limit_words(text, word_limit=200):
    words = text.split()
    if len(words) > word_limit:
        return ' '.join(words[:word_limit]) + '...'  # Add ellipsis if text is truncated
    return text

text_files_path = '/home/vcl3d/coco_dataset_VOX/train2014_desc'
text_files = [file for file in os.listdir(text_files_path) if file.endswith('_desc.txt')]
text_files = sorted(text_files)
# Create a dictionary to store text descriptions with image filenames (without extension) as keys
text_dict = {}

In [9]:
# Load text descriptions from each text file and match them with the images
for text_file in text_files:
    image_name = os.path.splitext(text_file)[0]
    text_file_path = os.path.join(text_files_path, text_file)

    with open(text_file_path, 'r') as file:
        text_descriptions = file.read().splitlines()

    text_dict[image_name] = text_descriptions

    text_descriptions = remove_numbers(text_descriptions)
    text_descriptions = limit_words(text_descriptions , word_limit=50)
    text_dict[image_name] = text_descriptions
# Convert image paths to strings by extracting the file name from the full path
image_filenames = [os.path.splitext(os.path.basename(image_path.filename))[0] for image_path in dataset["image"]]
text_filenames = [os.path.splitext(os.path.basename(image_path.filename))[0] + '_desc'  for image_path in dataset["image"]]
# Add the "text" column to the dimg dataset
dataset = dataset.add_column("caption", [text_dict[filename] for filename in text_filenames])
dataset = dataset.add_column("image_path", items_list)
dataset = dataset.add_column("height", heights_list)
dataset = dataset.add_column("width", widths_list)
dataset = dataset.add_column("image_id", image_id_list)
dataset = dataset.add_column("file_name", image_filenames)

In [10]:
#EVALUATION
val_text_files_path = '/home/vcl3d/coco_dataset_VOX/val2014_desc'
val_text_files = [file for file in os.listdir(val_text_files_path) if file.endswith('_desc.txt')]
val_text_files = sorted(val_text_files)

# Create a dictionary to store text descriptions with image filenames (without extension) as keys
val_text_dict = {}

# Load text descriptions from each text file and match them with the images
for text_file in val_text_files:
    val_image_name = os.path.splitext(text_file)[0]
    val_text_file_path = os.path.join(val_text_files_path, text_file)

    with open(val_text_file_path, 'r') as file:
        val_text_descriptions = file.read().splitlines()

    val_text_dict[val_image_name] = val_text_descriptions

    val_text_descriptions = remove_numbers(val_text_descriptions)
    val_text_descriptions = limit_words(val_text_descriptions , word_limit=50)
    val_text_dict[val_image_name] = val_text_descriptions

# Convert image paths to strings by extracting the file name from the full path
val_image_filenames = [os.path.splitext(os.path.basename(val_image_path.filename))[0] for val_image_path in val_dataset["image"]]
val_text_filenames = [os.path.splitext(os.path.basename(val_image_path.filename))[0] + '_desc'  for val_image_path in val_dataset["image"]]
# Add the "text" column to the dimg dataset
val_dataset = val_dataset.add_column("caption", [val_text_dict[filename] for filename in val_text_filenames])
val_dataset = val_dataset.add_column("image_path", val_items_list)
val_dataset = val_dataset.add_column("height", val_heights_list)
val_dataset = val_dataset.add_column("width", val_widths_list)
val_dataset = val_dataset.add_column("image_id", val_image_id_list)
val_dataset = val_dataset.add_column("file_name", val_image_filenames)

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
###UNCOMMENT TO SAVE TENSORS FROM DATASET
# #gtp version
from tqdm import tqdm
import numpy as np
import torch
from PIL import Image

def feature_extraction_fn(image_paths, tensor_name):
# Define your image_paths and feature_extractor

    from tqdm import tqdm
    import numpy as np
    import torch
    from PIL import Image

    # Define your image_paths and feature_extractor
    batch_size = 100 # Choose an appropriate batch size based on your system's memory capacity

    num_images = len(image_paths)
    num_batches = int(np.ceil(num_images / batch_size))

    inputs = []  # List to store processed image data

    with tqdm(total=num_batches, desc="Processing Batches") as pbar_batch:
        for i in range(0, num_images, batch_size):
            batch_paths = image_paths[i:i + batch_size]
            batch_images = []  # List to store images for the current batch
            
            # Process images in the current batch
            for image_file in batch_paths:
                        with Image.open(image_file) as img:
                            image_path = feature_extractor(img, return_tensors="np")
                            batch_images.append(image_path.pixel_values)
            # Convert the list of batch images to a numpy array
            batch_array = np.array(batch_images)
            
            # Convert the numpy array to a PyTorch tensor
            batch_tensor = torch.tensor(batch_array)
            
            # Append the batch tensor to the inputs list
            inputs.append(batch_tensor)
            pbar_batch.update(1)

    # Concatenate the list of batch tensors along the first dimension to create the final tensor
    pbar_batch.close()
    tensor = torch.cat(inputs, dim=0)
    file_name = f'{tensor_name}_tensor_data.pt'
    print(tensor.dtype)
    #tensor = torch.quantize_per_tensor(float_tensor, scale=0.2, zero_point=0, dtype=torch.qint32)
    tensor = tensor.half()
    torch.save(tensor, file_name)
    print(tensor.dtype)
    print("tensor saved in:", file_name)
    return tensor   

In [12]:
# split_index_1 = len(dataset) // 3
# split_index_2 = 2 * len(dataset) // 3
# # Split the dataset into three parts
# first_third = dataset[:split_index_1]
# second_third = dataset[split_index_1:split_index_2]
# third_third = dataset[split_index_2:]

In [13]:
#feature_extraction_fn(val_dataset["image_path"], tensor_name="f16val")

Processing Batches: 100%|██████████| 405/405 [03:08<00:00,  2.15it/s]


torch.float32
torch.float16
tensor saved in: f16val_tensor_data.pt


tensor([[[[[ 0.3333,  0.3491,  0.3726,  ...,  0.9922,  0.9922,  0.9922],
           [ 0.3254,  0.3411,  0.3647,  ...,  0.9922,  0.9922,  0.9922],
           [ 0.3411,  0.3491,  0.3726,  ...,  0.9922,  0.9922,  0.9922],
           ...,
           [-0.7803, -0.7803, -0.7803,  ...,  0.9136,  0.8979,  0.8667],
           [-0.7881, -0.7881, -0.7803,  ...,  0.9058,  0.8823,  0.8667],
           [-0.7881, -0.7881, -0.7881,  ...,  0.8979,  0.8745,  0.8511]],

          [[ 0.0196,  0.0353,  0.0745,  ...,  1.0000,  1.0000,  1.0000],
           [ 0.0118,  0.0275,  0.0667,  ...,  1.0000,  1.0000,  1.0000],
           [ 0.0275,  0.0353,  0.0588,  ...,  1.0000,  1.0000,  1.0000],
           ...,
           [-0.7256, -0.7256, -0.7334,  ...,  0.2627,  0.2471,  0.2313],
           [-0.7334, -0.7334, -0.7334,  ...,  0.2549,  0.2313,  0.2157],
           [-0.7334, -0.7334, -0.7412,  ...,  0.2313,  0.2235,  0.2000]],

          [[ 0.0275,  0.0196,  0.0039,  ...,  0.9609,  0.9609,  0.9609],
           [ 0.

In [14]:
#feature_extraction_fn(second_third["image_path"], tensor_name="train2")

In [15]:
#feature_extraction_fn(third_third["image_path"], tensor_name="train3")

In [16]:
# import torch
# import concurrent.futures

# # Define your function to process a batch of tensors
# def process_tensor_batch(batch_tensor):
#     # Perform your processing operations here
#     # Example: Multiply each element in the tensor by 2
#     processed_batch = batch_tensor * 1
#     return processed_batch

# # Load your tensor (example)
# tensor = torch.load('train1_tensor_data.pt')

# # Split the tensor into batches (adjust batch_size based on your needs)
# batch_size = 100
# max_workers = 10
# tensor_batches = [tensor[i:i + batch_size] for i in range(0, len(tensor), batch_size)]

# # Define a function to process a single batch in parallel
# def process_batch_parallel(batch):
#     with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
#         # Process the batch in parallel using ThreadPoolExecutor
#         processed_batch_list = list(executor.map(process_tensor_batch, batch))
#     return processed_batch_list

# # Process tensor batches in parallel
# processed_batches = process_batch_parallel(tensor_batches)

# # Concatenate the processed batches back into a single tensor
# processed_tensor = torch.cat([torch.tensor(batch) for batch in processed_batches], dim=0)

# # Now, 'processed_tensor' contains the processed data


In [17]:
loaded_tensor1 = torch.load('f16train1_tensor_data.pt')

In [18]:
loaded_tensor1.size()

torch.Size([27537, 1, 3, 224, 224])

In [19]:
loaded_tensor2 = torch.load('f16train2_tensor_data.pt')

In [20]:
loaded_tensor3 = torch.load('f16train3_tensor_data.pt')

In [21]:
import pandas as pd

# Assuming 'dataset' is your dataset object
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset)

# Save the DataFrame to a CSV file
df.to_csv('train_dataset.csv', index=False)

In [ ]:
import pandas as pd

# Assuming 'dataset' is your dataset object
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(val_dataset)

# Save the DataFrame to a CSV file
df.to_csv('val_dataset.csv', index=True)

In [20]:
from tqdm import tqdm
import numpy as np
# text preprocessing step
#gpt version
def tokenization_fn(captions, max_target_length):
    """Run tokenization on captions."""
    tokenized_labels = []
    # Initialize tqdm progress bar with the total number of captions
    with tqdm(total=len(captions), desc="Tokenizing Captions") as pbar:
        # Iterate through captions and tokenize them
        for caption in captions:
            # Tokenize the caption using the tokenizer
            tokens = tokenizer(caption, padding="max_length", max_length=max_target_length).input_ids
            # Append tokenized caption to the list
            tokenized_labels.append(tokens)
            pbar.update(1)  # Update the progress bar
    # Convert the list of tokenized labels to a PyTorch tensor
    labels = torch.tensor(tokenized_labels, dtype=torch.long)
    pbar.close()

    return labels

#original version
# def tokenization_fn(captions, max_target_length):
#     """Run tokenization on captions."""
#     labels = tokenizer(captions,
#                       padding="max_length",
#                       max_length=max_target_length).input_ids

#     return labels


# image preprocessing step
#original version
# def feature_extraction_fn(image_paths, check_image=True):
#     """
#     Run feature extraction on images
#     If `check_image` is `True`, the examples that fails during `Image.open()` will be caught and discarded.
#     Otherwise, an exception will be thrown.
#     """
#     inputs = []
#     counter=0

#     if check_image==False:
#         images = []
#         to_keep = []
#         for image_file in image_paths:
#             try:
#                 img = Image.open(image_file)
#                 #if img.mode != 'RGB':
#                    #img = img.convert('RGB')
#                 images.append(img)
#                 to_keep.append(True)               
#             except Exception:
#                 to_keep.append(False)
#     else:
#         #images = [Image.open(image_file) for image_file in image_paths]
#     #     for image_file in image_paths:
#     #         with Image.open(image_file) as test_image:
#     #             image_paths[counter] = feature_extractor(images=test_image, return_tensors="np")
#     #             counter+=1
#     # encoder_inputs = feature_extractor(images=image_paths, return_tensors="np")
#     # return encoder_inputs.pixel_values
#         with tqdm(total=len(image_paths), desc="Processing Images") as pbar:  
#             for image_file in image_paths:
#                 #with Image.open(image_file) as test_image:
#                 img = Image.open(image_file)
#                 image_path = feature_extractor(img, return_tensors="np")
#                 inputs.append(image_path.pixel_values)
#                 img.close()
#                 pbar.update(1) 
#     numpy_array = np.array(inputs)
#     tensor = torch.tensor(numpy_array)
#     #tensor = torch.tensor(inputs, dtype=torch.float32)            
#     return tensor            

#     #return encoder_inputs.pixel_values

def preprocess_fn(examples, max_target_length, tensor):
    """Run tokenization + image feature extraction"""
    #image_paths = examples['image_path']
    captions = examples['caption']
    model_inputs = {}
    # This contains image path column
    model_inputs['labels'] = tokenization_fn(captions, max_target_length)
    #model_inputs['pixel_values'] = feature_extraction_fn(image_paths)
    model_inputs['pixel_values'] = tensor#torch.load(tensor)
    return model_inputs


In [21]:
#y = tokenization_fn(val_dataset["caption"], 100)

In [22]:
#preprocess_fn(examples=val_dataset, max_target_length=100, tensor=loaded_tensor)


In [23]:
tensor = torch.cat((loaded_tensor1, loaded_tensor2, loaded_tensor3), dim=0)

In [26]:
tensor[0]

In [16]:
###gpt version
progress_bar = [tqdm(total=len(dataset), desc="Processing Validation Dataset")]

# Define a function to preprocess a single example and update the progress bar
def process_example(example):
    processed_example = preprocess_fn(example, max_target_length=200, tensor=tensor)
    progress_bar[0].update(1)  # Update the progress bar after processing each example
    return processed_example

# Apply the preprocess_fn to the validation dataset using map
processed_dataset = dataset.map(
    function=process_example,
    batched=True,
    #remove_columns=val_dataset.column_names,
)

# Close the progress bar after processing is complete
progress_bar[0].close()
###original version***    
# val_processed_dataset = val_dataset.map(
#     function=preprocess_fn(val_dataset, max_target_length=200, tensor=loaded_tensor),
#     batched=True,
#     fn_kwargs={"max_target_length": 200},
#     #remove_columns=dataset.column_names
# )

Processing Validation Dataset:   0%|          | 0/82611 [00:00<?, ?it/s]Parameter 'function'=<function process_example at 0x7fbd337e7520> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Processing Validation Dataset:   0%|          | 1/82611 [00:09<215:52:33,  9.41s/it]

KeyboardInterrupt: 

In [ ]:
preprocess_fn(dataset, 15, check_image = True)

processed_dataset = dataset.map(
    function=preprocess_fn,
    batched=True,
    fn_kwargs={"max_target_length": 200},
    #remove_columns=val_dataset.column_names
)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    output_dir="./image-captioning-output-107epochs",
    num_train_epochs= 20
)


In [ ]:
import evaluate
metric = evaluate.load("rouge")


In [ ]:
import numpy as np

ignore_pad_token_for_loss = True


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds,
                                                     decoded_labels)

    result = metric.compute(predictions=decoded_preds,
                            references=decoded_labels,
                            use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    return result


In [ ]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=val_processed_dataset,
    #eval_dataset=val_updated_dataset,
    eval_dataset=val_processed_dataset,
    #train_dataset=processed_dataset['train'],
    #eval_dataset=processed_dataset['validation'],
    data_collator=default_data_collator,
)

trainer.compute_metrics

trainer.train()

trainer.save_model("./image-captioning-output-107epochs")

tokenizer.save_pretrained("./image-captioning-output-107epochs")

from transformers import pipeline
image_captioner = pipeline("image-to-text", model="./image-captioning-output-107epochs", max_new_tokens=10)
dataset["image"][5]
image_captioner(dataset['image'][5])
image_captioner("test_images/COCO_test2015_000000000014.jpg")

